In [13]:
from utils.find_miner_id import ReadTokens
import json
from datetime import datetime


In [33]:
def calculate_pplns_participation(shares_data, block_data, n_factor):
    # Sort shares by blockheight in descending order
    shares_data.sort(key=lambda x: x['blockheight'], reverse=True)

    # Get block information
    block_height = block_data[0]['blockheight']
    network_difficulty = block_data[0]['networkdifficulty']

    
    target_shares = network_difficulty * n_factor
    print(network_difficulty, 'net diff', target_shares, 'target shares')

    # Initialize variables
    total_shares = 0
    valid_shares = []
    miner_shares = {}

    # Collect shares until we reach or exceed the target
    for share in shares_data:
        
        if share['blockheight'] > block_height:
            continue

        valid_shares.append(share)
        total_shares += share['difficulty']

        miner = share['miner']
        if miner not in miner_shares:
            miner_shares[miner] = 0
        miner_shares[miner] += share['difficulty']

        if total_shares >= target_shares:
            print(share, 'share')
            break

    # Calculate participation percentages
    participation = {miner: shares / total_shares for miner, shares in miner_shares.items()}

    return participation, total_shares



In [34]:
reader = ReadTokens()

# Load share data
shares_data = reader.get_api_data('http://188.245.66.57:8000/miningcore/shares')

# Load block data
block_data = reader.get_api_data('http://188.245.66.57:8000/miningcore/blocks')

# Set the N factor (this could be user-defined)
n_factor = 0.5  # For example, 2 times the network difficulty

# Calculate participation
participation, total_shares = calculate_pplns_participation(shares_data, block_data, n_factor)

# Print results
print(f"Participation for block {block_data[0]['blockheight']} (N factor: {n_factor}):")
print('TOTAL SHARES: {}'.format(total_shares))
for miner, percentage in participation.items():
    print(f"Miner {miner}: {percentage:.2%}")

# Optional: Calculate rewards
block_reward = block_data[0]['reward']
print(f"\nReward distribution for block reward of {block_reward}:")
for miner, percentage in participation.items():
    miner_reward = block_reward * percentage
    print(f"Miner {miner}: {miner_reward:.8f}")

208348.859402613 net diff 104174.4297013065 target shares
Participation for block 1357953 (N factor: 0.5):
TOTAL SHARES: 396.410631230505
Miner 9hDaPXB2Tx5BHQfiqcELp6m3wskR4g8sa5Tna4G9y4TeMa878n9: 2.09%
Miner 9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk: 76.67%
Miner 9eg7v2nkypUZbdyvSKSD9kg8FNwrEdTrfC2xdXWXmEpDAFEtYEn: 21.24%

Reward distribution for block reward of 24.0:
Miner 9hDaPXB2Tx5BHQfiqcELp6m3wskR4g8sa5Tna4G9y4TeMa878n9: 0.50132499
Miner 9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk: 18.40004837
Miner 9eg7v2nkypUZbdyvSKSD9kg8FNwrEdTrfC2xdXWXmEpDAFEtYEn: 5.09862664


In [32]:
block_data = reader.get_api_data('http://188.245.66.57:8000/miningcore/blocks')
block_data

[{'id': 1,
  'poolid': 'Shark-Dev-Pool',
  'blockheight': 1357953,
  'networkdifficulty': 208348.859402613,
  'status': 'confirmed',
  'type': None,
  'confirmationprogress': 1.0,
  'effort': 1.8476115899233763,
  'transactionconfirmationdata': '40016c28e23707f6',
  'miner': '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk',
  'reward': 24.0,
  'source': 'Shark-Dev-Pool',
  'hash': 'adca97da6d5e1295e8e942b999adda22066cee0a892afdb965cb703aee0c3971',
  'created': '2024-09-22T21:15:36.427181+00:00'}]

In [28]:
shares_data = reader.get_api_data('http://188.245.66.57:8000/miningcore/shares')


[{'poolid': 'Shark-Dev-Pool',
  'blockheight': 1358026,
  'difficulty': 0.04513937035067354,
  'networkdifficulty': 208348.859402613,
  'miner': '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk',
  'worker': 'GRAYSPEAK',
  'useragent': 'lolMiner 1.90',
  'ipaddress': '::ffff:216.147.122.189',
  'source': 'Shark-Dev-Pool',
  'created': '2024-09-23T00:04:42.124911+00:00'},
 {'poolid': 'Shark-Dev-Pool',
  'blockheight': 1358026,
  'difficulty': 0.00363794921875,
  'networkdifficulty': 208348.859402613,
  'miner': '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk',
  'worker': 'LAPLATAPEAK',
  'useragent': 'teamredminer/0.10.21',
  'ipaddress': '::ffff:216.147.122.189',
  'source': 'Shark-Dev-Pool',
  'created': '2024-09-23T00:04:54.044433+00:00'},
 {'poolid': 'Shark-Dev-Pool',
  'blockheight': 1358026,
  'difficulty': 0.00363794921875,
  'networkdifficulty': 208348.859402613,
  'miner': '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk',
  'worker': 'LAPLATAPEAK',
  'useragent':

In [29]:
shares_data.sort(key=lambda x: x['blockheight'], reverse=True)

In [31]:
shares_data

[{'poolid': 'Shark-Dev-Pool',
  'blockheight': 1359491,
  'difficulty': 0.0072654857565560985,
  'networkdifficulty': 268692.902627631,
  'miner': '9eg7v2nkypUZbdyvSKSD9kg8FNwrEdTrfC2xdXWXmEpDAFEtYEn',
  'worker': 'PIKESPEAK',
  'useragent': 'teamredminer/0.10.21',
  'ipaddress': '::ffff:216.147.125.127',
  'source': 'Shark-Dev-Pool',
  'created': '2024-09-25T00:03:59.894943+00:00'},
 {'poolid': 'Shark-Dev-Pool',
  'blockheight': 1359491,
  'difficulty': 0.012527211451163341,
  'networkdifficulty': 268692.902627631,
  'miner': '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk',
  'worker': 'GRAYSPEAK',
  'useragent': 'lolMiner 1.90',
  'ipaddress': '::ffff:216.147.125.127',
  'source': 'Shark-Dev-Pool',
  'created': '2024-09-25T00:04:00.210809+00:00'},
 {'poolid': 'Shark-Dev-Pool',
  'blockheight': 1359491,
  'difficulty': 0.012527211451163341,
  'networkdifficulty': 268692.902627631,
  'miner': '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk',
  'worker': 'GRAYSPEAK',
  'usera

In [16]:
a = 77.8
b = 2.12
c = 21.56

total_shares = a + b + c

In [17]:
a / total_shares, b / total_shares, c / total_shares

(0.7666535277887268, 0.020890815924320065, 0.21245565628695307)